Imports

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tqdm import tqdm
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Model

**File Directory**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
base_dir = '/content/drive/MyDrive/Capstone_In-Craft/Dataset/'

**Dataset Generator**

In [ ]:
IMAGE_SIZE=224
BATCH_SIZE=32

#pre=processing
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.2
)

train_datagen=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    #class_mode = 'sparse',
    class_mode = 'categorical',
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    #class_mode = 'sparse',
    class_mode = 'categorical',
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 1209 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


**Transfer Learning**

In [ ]:
#Full feature
#MODULE_HANDLE = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4'

#module = hub.load(MODULE_HANDLE)

In [ ]:
#Less Feature

local_weights_file = '/content/drive/MyDrive/Capstone_In-Craft/TF_weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'

In [ ]:
def create_pre_trained_model(local_weights_file):
  """
  Initializes an InceptionV3 model.
  
  Args:
    local_weights_file (string): path pointing to a pretrained weights H5 file
    
  Returns:
    pre_trained_model: the initialized InceptionV3 model
  """

  pre_trained_model = MobileNetV2(input_shape = (224, 224, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  # Make all the layers in the pre-trained model non-trainable
  for layer in pre_trained_model.layers:
    layer.trainable = False


  return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)

# Print the model summary
pre_trained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
def output_of_last_layer(pre_trained_model):
  """
  Gets the last layer output of a model
  
  Args:
    pre_trained_model (tf.keras Model): model to get the last layer output from
    
  Returns:
    last_output: output of the model's last layer 
  """

  last_desired_layer = pre_trained_model.get_layer('block_14_project_BN')
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)


  return last_output



In [ ]:
last_output = output_of_last_layer(pre_trained_model)

last layer output shape:  (None, 7, 7, 160)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 160), dtype=tf.float32, name=None), name='block_14_project_BN/FusedBatchNormV3:0', description="created by layer 'block_14_project_BN'")


Model

In [ ]:
#Full Feature

#model=tf.keras.Sequential()

#model.add(hub.KerasLayer(MODULE_HANDLE,input_shape=(224,224,3)))

#model.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=2,kernel_size=3,activation='relu'))
#model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

#model.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
#model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

#model.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
#model.add(tf.keras.layers.MaxPool2D(pool_size=2))

#model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(1024,activation='relu'))
#model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [ ]:
def create_final_model(pre_trained_model, last_output):
  """
  Appends a custom model to a pre-trained model
  
  Args:
    pre_trained_model (tf.keras Model): model that will accept the train/test inputs
    last_output (tensor): last layer output of the pre-trained model
    
  Returns:
    model: the combined model
  """
  # Flatten the output layer to 1 dimension
  x = tf.keras.layers.Flatten()(last_output)

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = tf.keras.layers.Dense(1024, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = tf.keras.layers.Dropout(0.2)(x)   
  # Add a final sigmoid layer for classification
  x = tf.keras.layers.Dense(10, activation='softmax')(x)        

  # Create the complete model by using the Model class
  model = Model(inputs=pre_trained_model.input, outputs=x)

  # Compile the model
  #optimizer: tf.keras.optimizers.RMSprop(learning_rate=0.002), tf.keras.optimizers.Adam()
  #loss: tf.keras.losses.CategoricalCrossentropy(), tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0005), 
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics = ['accuracy'])


  
  return model

In [ ]:
#model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model = create_final_model(pre_trained_model, last_output)

In [ ]:
model.fit(train_datagen,epochs=15,validation_data=test_datagen)

Epoch 1/15
38/38 [==============================] - 1005s 26s/step - loss: 4.8480 - accuracy: 0.6253 - val_loss: 0.8627 - val_accuracy: 0.8133
Epoch 2/15
38/38 [==============================] - 401s 11s/step - loss: 0.8544 - accuracy: 0.8189 - val_loss: 0.3952 - val_accuracy: 0.8767
Epoch 3/15
38/38 [==============================] - 398s 11s/step - loss: 0.6742 - accuracy: 0.8495 - val_loss: 1.1341 - val_accuracy: 0.8800
Epoch 4/15
38/38 [==============================] - 412s 11s/step - loss: 0.4917 - accuracy: 0.8908 - val_loss: 0.2870 - val_accuracy: 0.9067
Epoch 5/15
38/38 [==============================] - 406s 11s/step - loss: 0.3434 - accuracy: 0.9189 - val_loss: 0.5140 - val_accuracy: 0.8767
Epoch 6/15
38/38 [==============================] - 410s 11s/step - loss: 0.3184 - accuracy: 0.9222 - val_loss: 0.5341 - val_accuracy: 0.8900
Epoch 7/15
38/38 [==============================] - 406s 11s/step - loss: 0.3267 - accuracy: 0.9305 - val_loss: 0.5321 - val_accuracy: 0.8767
Epoch

Convert model to tflite

In [ ]:
RPS_SAVED_MODEL = "rps_saved_model"

In [ ]:
tf.saved_model.save(model, RPS_SAVED_MODEL)

In [ ]:
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_input_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


2023-06-01 03:53:42.523852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
loaded = tf.saved_model.load(RPS_SAVED_MODEL)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(RPS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = '19.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)